In [1]:
import json, os, pickle, random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from utils import get_backend_properties_v1
from mlp import MLP1, MLP2, MLP3, encode_data

import matplotlib.pyplot as plt
import seaborn as sns
from noise_utils import AddNoise, RemoveReadoutErrors
from mlp import recursive_dict_loop, count_gates_by_rotation_angle
from mbd_utils import calc_imbalance
import torch
from torch import nn
from sklearn import datasets
import sklearn
from utils import circuit_to_graph_data_json
from utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
)
from qiskit.circuit.random import random_circuit
from sklearn.model_selection import train_test_split
from qiskit.converters import circuit_to_dag
from scripts.data_setup import load_data
from scripts.gate_dict_script import gate_dict_method

In [2]:
train_circuits, train_ideal_exp_vals, train_noisy_exp_vals, test_circuits, test_ideal_exp_vals, test_noisy_exp_vals = load_data('data/data_1_0')

Data loaded successfully!


In [3]:
backend = FakeLima()
gate_dictionary = gate_dict_method()
backend_properties = get_backend_properties_v1(backend)

In [4]:
def get_qc_operations(qc):
    qc = transpile(circuits=qc, 
               backend=backend, 
               optimization_level=0)
    operations = []
    for operation in qc.data:
        operations.append(operation)
    return operations

In [5]:
# each word is -> gatetype, params, t1, t2, readout_error, gate_error, gate_length

def operations_to_features(circuits, true_exp_vals, n_qubits):
    X = []
    i = 0
    for qc in circuits:
        #dag_circuit = circuit_to_dag(qc)
        circuit_instructions = get_qc_operations(qc)
        # CircuitInstruction(operation=Instruction(name='rz', num_qu .... :)))))))
        features = []
        for i in range(n_qubits):
            features.append([])
        # qubit errors 
        for qubit_index in range(n_qubits):
            t1 = backend_properties['qubits_props'][qubit_index]['t1']
            t2 = backend_properties['qubits_props'][qubit_index]['t2']
            readout_error = backend_properties['qubits_props'][qubit_index]['readout_error']
            features[qubit_index].append([0, 0, t1, t2, readout_error, 0, 0])
        for circuit_instruction in circuit_instructions:
            gate_instruction = circuit_instruction.operation # the quantum gate applied to a qubit/qubits
            op_name = gate_instruction.name
            #print(op_name)
            op_params = gate_instruction.params

            if len(op_params) == 0:
                op_params = float('0')
            else:
                op_params = op_params[0]

            #print(gate_dict[op_name])

            op_encoded = gate_dictionary[op_name]
            op_qubit = circuit_instruction.qubits
            op_qubit_index = op_qubit[0].index

            if op_name == 'cx':
                op_gate_quibts = circuit_instruction.qubits
                backend_op_name = f'{op_name}_{op_gate_quibts[0].index}_{op_gate_quibts[1].index}'
            else:
                backend_op_name = f'{op_name}_{op_qubit_index}'

            gate_error = backend_properties['gate_props'][backend_op_name]['gate_error']
            gate_length = backend_properties['gate_props'][backend_op_name]['gate_length']
            'sx_1'

            qubit_feature = [op_encoded, op_params, 0, 0, 0, gate_error, gate_length]
            features[op_qubit_index].append(qubit_feature)
            #print(f'{op_encoded} | {op_params} | {op_qubit}')

        max_length = max(len(sublist) for sublist in features)
        for sublist in features:
            while len(sublist) < max_length:
                sublist.append([float('0'), float('0'), float('0'), float('0'), float('0'), float('0'), float('0')])

        X.append(features)

    y = []
    for exp_val in true_exp_vals:
        y.append(exp_val)
        
    return X, y

In [6]:
X_train, y_train = operations_to_features(train_circuits, train_ideal_exp_vals, 5)
X_test, y_test = operations_to_features(test_circuits, test_ideal_exp_vals, 5)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
import numpy as np

def evaluate_model(rnn_extractor, predictor, criterion, X, y):
    with torch.no_grad():
        all_predictions = []
        all_losses = []
        for i in range(len(X)):
            circ_features_tensor = torch.FloatTensor(X[i])
            rnn_output = rnn_extractor(circ_features_tensor)
            rnn_output = torch.flatten(rnn_output)
            noisy_exp_val = torch.from_numpy(np.array(np.float32(test_noisy_exp_vals[i])))
            rnn_output_noisy_exp_val = torch.cat((rnn_output, noisy_exp_val.reshape(1)))
            pred = predictor(rnn_output_noisy_exp_val)
            loss = criterion(pred, torch.FloatTensor([y[i]]))
            
            all_predictions.append(pred.item())
            all_losses.append(loss.item())
        
        rmse = np.sqrt(np.mean(np.square(np.array(all_predictions) - np.array(y))))

        #print(rnn_extractor.parameters())
        
        return rmse



In [9]:
n_qubits = 5

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# remake this because its not working as intented i think, play with dropout as it does a lot

class RNNFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNFeatureExtractor, self).__init__()
        self.qubit_rnns = nn.ModuleDict()
        for i in range(n_qubits):
            self.qubit_rnns[f'rnn_{i}'] = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0.1)

        
    def forward(self, x):
        rnn_outputs = []
        #print(x.shape)
        for rnn in self.qubit_rnns.values():
            rnn_output, _ = rnn(x)
            rnn_output_last = rnn_output[:, -1, :]
            rnn_outputs.append(rnn_output_last)     
        
        output = torch.stack(rnn_outputs, dim=1)
        #print(output.shape)
        #print(1 + '3')
        return output


class ExpectationValuePredictor(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               hidden_layers: int,
               output_shape: int,
               p_drouput: int):
    super().__init__()
    self.layers = nn.ModuleDict()
    self.nLayers = hidden_layers

    self.layers['input'] = nn.Linear(input_shape * n_qubits + 1,
                                     hidden_units)

    for i in range(hidden_layers):
      self.layers[f'hidden{i}'] = nn.Linear(hidden_units,
                                            hidden_units)

    self.layers['output'] = nn.Linear(hidden_units,
                                      output_shape)
    
    self.dropout = p_drouput

  def forward(self, x):
    #print(x.shape)
    x = torch.flatten(x)
    ReLU = nn.ReLU()

    x = ReLU(self.layers['input'](x))

    #print(x.shape)

    for i in range(self.nLayers):
        x = ReLU(self.layers[f'hidden{i}'](x))
        x = torch.nn.functional.dropout(x, p=self.dropout, training=self.training)

    x = self.layers['output'](x)

    return x

def train_and_evaluate_model(rnn_extractor, predictor, criterion, optimizer, X_train, y_train, X_test, y_test, num_epochs):
    for epoch in range(num_epochs):
        rnn_extractor.train()
        predictor.train()
        train_losses = []
        for i in range(len(X_train)):
            circ_features_tensor = torch.FloatTensor(X_train[i])
            rnn_output = rnn_extractor(circ_features_tensor)
            rnn_output = torch.flatten(rnn_output)
            noisy_exp_val = torch.from_numpy(np.array(np.float32(train_noisy_exp_vals[i])))
            rnn_output_noisy_exp_val = torch.cat((rnn_output, noisy_exp_val.reshape(1)))
            pred = predictor(rnn_output_noisy_exp_val)
            loss = criterion(pred, torch.FloatTensor([y_train[i]]))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        
        rnn_extractor.eval()
        predictor.eval()
        test_mse = evaluate_model(rnn_extractor, predictor, criterion, X_test, y_test)

        if epoch % 1 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, "
                f"Train Loss: {train_loss:.4f}, "
                f"Test RMSE: {test_mse:.4f}")
            
        #print(rnn_extractor.state_dict())

input_size = 7
hidden_size = 8
num_layers = 4
output_size = 1

rnn_extractor = RNNFeatureExtractor(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
predictor = ExpectationValuePredictor(input_shape=(hidden_size * n_qubits), hidden_units=64, hidden_layers=2, output_shape=output_size, p_drouput=0.1)

criterion = nn.MSELoss()
optimizer = optim.Adam(list(predictor.parameters()) + list(rnn_extractor.parameters()), lr=0.001)

num_epochs = 5
train_and_evaluate_model(rnn_extractor, predictor, criterion, optimizer, X_train, y_train, X_test, y_test, num_epochs)


Epoch 1/5, Train Loss: 0.2979, Test RMSE: 0.3205
Epoch 2/5, Train Loss: 0.1712, Test RMSE: 0.2911
Epoch 3/5, Train Loss: 0.1661, Test RMSE: 0.3663
Epoch 4/5, Train Loss: 0.1609, Test RMSE: 0.2919
Epoch 5/5, Train Loss: 0.1513, Test RMSE: 0.3632


In [11]:
num_epochs = 8
optimizer = optim.Adam(list(predictor.parameters()) + list(rnn_extractor.parameters()), lr=0.0001)
train_and_evaluate_model(rnn_extractor, predictor, criterion, optimizer, X_train, y_train, X_test, y_test, num_epochs)

Epoch 1/8, Train Loss: 0.1208, Test RMSE: 0.2825
Epoch 2/8, Train Loss: 0.1234, Test RMSE: 0.2818
Epoch 3/8, Train Loss: 0.1173, Test RMSE: 0.2896
Epoch 4/8, Train Loss: 0.1180, Test RMSE: 0.2973
Epoch 5/8, Train Loss: 0.1172, Test RMSE: 0.2834
Epoch 6/8, Train Loss: 0.1134, Test RMSE: 0.2888
Epoch 7/8, Train Loss: 0.1159, Test RMSE: 0.2907
Epoch 8/8, Train Loss: 0.1126, Test RMSE: 0.2776


In [12]:
def fix_random_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    print(f'random seed fixed to {seed}')

In [13]:
properties = get_backend_properties_v1(backend=backend)

In [16]:
def encode_data(circuits, properties, ideal_exp_vals, noisy_exp_vals, num_qubits, meas_bases=None):
    # if isinstance(noisy_exp_vals[0], list) and len(noisy_exp_vals[0]) == 1:
    #     noisy_exp_vals = [x[0] for x in noisy_exp_vals]

    gates_set = sorted(properties['gates_set'])     # must sort!

    if meas_bases is None:
        meas_bases = [[]]

    vec = [np.mean(recursive_dict_loop(properties, out=[], target_key1='cx', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='id', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='sx', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='x', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='rz', target_key2='gate_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='readout_error'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='t1'))]
    vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='t2'))]
    vec = torch.tensor(vec) * 100  # put it in the same order of magnitude as the expectation values

    bin_size = 0.1 * np.pi
    num_angle_bins = int(np.ceil(4 * np.pi / bin_size))

    X = torch.zeros([len(circuits), len(vec) + len(gates_set) + num_angle_bins + num_qubits + len(meas_bases[0])])

    vec_slice = slice(0, len(vec))
    gate_counts_slice = slice(len(vec), len(vec)+len(gates_set))
    angle_bins_slice = slice(len(vec)+len(gates_set), len(vec)+len(gates_set)+num_angle_bins)
    exp_val_slice = slice(len(vec)+len(gates_set)+num_angle_bins, len(vec)+len(gates_set)+num_angle_bins+num_qubits)
    meas_basis_slice = slice(len(vec)+len(gates_set)+num_angle_bins+num_qubits, len(X[0]))

    X[:, vec_slice] = vec[None, :]

    for i, circ in enumerate(circuits):
        gate_counts_all = circ.count_ops()
        X[i, gate_counts_slice] = torch.tensor(
            [gate_counts_all.get(key, 0) for key in gates_set]
        ) * 0.01  # put it in the same order of magnitude as the expectation values

    for i, circ in enumerate(circuits):
        gate_counts = count_gates_by_rotation_angle(circ, bin_size)
        X[i, angle_bins_slice] = torch.tensor(gate_counts) * 0.01  # put it in the same order of magnitude as the expectation values

        # if num_qubits > 1: assert len(noisy_exp_vals[i]) == num_qubits
        # elif num_qubits == 1: assert isinstance(noisy_exp_vals[i], float)

        X[i, exp_val_slice] = torch.tensor(noisy_exp_vals[i])

    if meas_bases != [[]]:
        assert len(meas_bases) == len(circuits)
        for i, basis in enumerate(meas_bases):
            X[i, meas_basis_slice] = torch.tensor(basis)

    y = torch.tensor(ideal_exp_vals, dtype=torch.float32)

    return X, y


In [17]:
X_train_rf, y_train_rf = encode_data(train_circuits, properties, train_ideal_exp_vals, train_noisy_exp_vals, num_qubits=n_qubits)
X_test_rf, y_test_rf = encode_data(test_circuits, properties, test_ideal_exp_vals, test_noisy_exp_vals, num_qubits=n_qubits)
BATCH_SIZE = 32
fix_random_seed(42)
train_dataset = TensorDataset(torch.Tensor(X_train_rf), torch.Tensor(y_train_rf))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = TensorDataset(torch.Tensor(X_test_rf), torch.Tensor(y_test_rf))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE*1000, shuffle=False)

random seed fixed to 42


In [18]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
rfr = RandomForestRegressor(n_estimators=300)
rfr.fit(X_train_rf, y_train_rf)

RandomForestRegressor(n_estimators=300)

In [19]:
from sklearn.metrics import root_mean_squared_error

In [20]:
y_pred_rf = rfr.predict(X_test_rf)
rms = root_mean_squared_error(y_test_rf, y_pred_rf)

In [21]:
rms

0.3254861534182519